In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

import sys
sys.path.append('../')
import plotting
import model

colormap = {
    "#00835": "#de2d26",
    "#00006": "#444444",
    "#01634": "#3182bd",
}

colormap_light = {
    "#00835": "#fcae91",
    "#00006": "#cccccc",
    "#01634": "#bdd7e7",
}

# Read abundance data from GCall

In [ ]:
pool_df = model.read_csvs("../data/internal_datasets/GCall/", keep_all_sequences=True)
pool_df['seq_id'] = pool_df.index.to_series().apply(lambda x: "#" + x[1:])
pool_df.drop(pool_df.loc[~pool_df.seq_id.isin(list(colormap.keys()))].index, inplace=True)
pool_df

# melt the dataframe
pool_df = pool_df.melt(id_vars="seq_id")
pool_df['n_cycles'] = pool_df.variable.apply(lambda x: float(15*int(x[-1])))
pool_df

# Merge with parameters and select representative sequences

In [ ]:
params = pd.read_csv("../data/internal_datasets/GCall/params.csv", dtype={"seq_id": str})
params['seq_id'] = params.seq_id.apply(lambda x: "#" + x[1:])
params.drop(params.loc[~params.seq_id.isin(list(colormap.keys()))].index, inplace=True)
params

# Calculate model predictions for the selected sequences

In [ ]:
x = np.arange(0, 90+1, 1)
y = {}

for seq_id in params.seq_id.unique():
    x0, eff = params.loc[params.seq_id == seq_id, "x0"].values[0], params.loc[params.seq_id == seq_id, "eff"].values[0]
    y[seq_id] = [x0 * (eff)**float(c) for c in x]

model_df = pd.DataFrame({"n_cycles": x})
for seq_id in y:
    model_df[seq_id] = y[seq_id]

# melt the dataframe
model_df = model_df.melt(id_vars="n_cycles")
model_df

# Small figure for main text

In [ ]:
fig = px.scatter(
    pool_df,
    x="n_cycles",
    y="value",
    color="seq_id",
    color_discrete_map=colormap,
)

fig.add_traces(
    px.line(
        model_df,
        x="n_cycles",
        y="value",
        color="variable",
        color_discrete_map=colormap_light,
    ).data[:]
)
fig.data = fig.data[::-1]


fig.add_annotation(
    x=0,
    y=1.6,
    text=f"x<sub>i</sub>(0)",
    showarrow=True,
    font=dict(color="black"),
    arrowhead=2,
    arrowsize=1,
    ax=20,
    ay=-20,
)

fig.add_annotation(
    x=60,
    y=2.1,
    text=f"ε<sub>i</sub>",
    showarrow=False,
    font=dict(color="black"),
)

fig.update_layout(
    xaxis_title="PCR cycles",
    yaxis_title="Relative coverage",
    margin=dict(l=0, r=5, t=10, b=20),
    height=75,
    width=110,
    showlegend=False,
)

fig.update_traces(
    marker_size=4,
)

fig.update_yaxes(
    range=[0, 2.5],
    dtick=1,
    minor_dtick=0.5,
)

fig.update_xaxes(
    range=[0, 95],
    dtick=15,
    minor_dtick=15,
)

fig.update_layout(
    template="simple_white",
    font_family="Inter",
    legend_font_size=20/3,
)
fig.update_yaxes(
    minor_ticks="outside", 
    title_font_family="Inter", 
    title_font_size=20/3, 
    tickfont_size=20/3, 
)
fig.update_xaxes(
    minor_ticks="outside", 
    title_font_family="Inter", 
    title_font_size=20/3, 
    tickfont_size=20/3, 
)
fig.for_each_annotation(lambda a: a.update(
    font_size=20/3,
    font_family="Inter",
))

fig.show()
fig.write_image("./figure_1/model_small.svg")

# Large figure for SI

In [ ]:
colormap = {
    "#00835": "#de2d26",
    "#00006": "#444444",
    "#01634": "#31a354",
}

colormap_light = {
    "#00835": "#fcae91",
    "#00006": "#cccccc",
    "#01634": "#bae4b3",
}


fig = px.scatter(
    pool_df,
    x="n_cycles",
    y="value",
    color="seq_id",
    color_discrete_map=colormap,
)

fig.add_traces(
    px.line(
        model_df,
        x="n_cycles",
        y="value",
        color="variable",
        color_discrete_map=colormap_light,
    ).data[:]
)
fig.data = fig.data[::-1]


fig.add_annotation(
    x=0,
    y=1.6,
    text=f"x<sub>i</sub>(0)",
    showarrow=True,
    font=dict(color="black"),
    arrowhead=2,
    arrowsize=1,
    ax=20,
    ay=-20,
)

fig.add_annotation(
    x=60,
    y=2.1,
    text=f"ε<sub>i</sub>",
    showarrow=False,
    font=dict(color="black"),
)

fig.update_layout(
    xaxis_title="PCR cycles",
    yaxis_title="Relative coverage",
    margin=dict(l=0, r=5, t=10, b=20),
    height=175,
    width=225,
    showlegend=False,
)

fig.update_traces(
    marker_size=6,
)

fig.update_yaxes(
    range=[0, 2.5],
    dtick=1,
    minor_dtick=0.5,
)

fig.update_xaxes(
    range=[0, 95],
    dtick=15,
    minor_dtick=15,
)


fig = plotting.standardize_plot(fig)
fig.show()
fig.write_image("./SI_figure_pcr_model_explanation/model_large.svg")